# Verification - Leftist Heap
###### (https://dodisturb.me/posts/2019-10-03-Verifying-the-Titular-Properties-of-a-Leftist-Heap.html)

A leftist heap is like a regular heap in that it has the heap invariant whereby the two child elements of node have labels that are less than the it's own label.

Leftist heaps are implemented as explicit binary trees whereas heaps are implemented with arrays.

The additional property of the leftist heap is that the left branch of a node has a size (rank) that is guaranteed to be greater than or equal to the right branch. This yields better asymptotics for merge operations because the left branches don't need to be rebuit, only the smaller right branches.

We first define a generic interface for heaps

In [22]:
:ext TypeFamilies

class Heap heap where
  {-# MINIMAL
    isEmpty, empty,
    (singleton | insert),
    (fromList | (singleton, merge)),
    (insert | (singleton, merge)),
    (merge | (decompose, insert)),
    (decompose | (findMax, deleteMax)) #-}
    
  type Elem heap
    
  -- Predicates
  isEmpty :: heap -> Bool
    
  -- Access
  findMax :: heap -> Maybe (Elem heap)
  findMax = fmap fst . decompose
    
  -- Creation
  empty :: heap
    
  singleton :: Elem heap -> heap
  singleton = flip insert empty
    
  fromList :: [Elem heap] -> heap
  fromList xs = -- O(n) for leftist heaps
      case go (map singleton xs) of
        [heap] -> heap
        []     -> empty
        _      -> error "Impossible. Did not converge to a single heap"
      where
      go [] = []
      go [x] = [x]
      go (x : y : rest) = go (merge x y : go rest)
      -- ^ this is much faster than the nieve insertion fold implementation. Has linear performance.
        
  -- Modification
  insert :: Elem heap -> heap -> heap
  insert x = merge (singleton x)
    
  merge :: heap -> heap -> heap
  merge heap1 heap2 =
      case decompose heap1 of
        Just (heapMax, heapRest) -> heapRest `merge` insert heapMax heap2
        Nothing -> heap2
            
  decompose :: heap -> Maybe (Elem heap, heap)
  decompose heap = (,) <$> findMax heap <*> deleteMax heap
    
  deleteMax :: heap -> Maybe heap
  deleteMax = fmap snd . decompose

Here's the simple list based version

In [23]:
instance Ord a => Heap [a] where
  type Elem [a] = a
  
  empty = []
  
  fromList = id
  
  insert = (:)
  
  merge = (++)
  
  decompose [] = Nothing
  decompose xs = Just (heapMax, left ++ tail right)
    where heapMax = maximum xs
          (left, right) = span (/= heapMax) xs

A leftist heap is tree and we'll implement it as such

In [24]:
data LeftistHeap a = Leaf | Node a Int (LeftistHeap a) (LeftistHeap a)

The `Int` paramter is the rank of the node which is the least distance to a `Leaf`. It is one more than the minimum of the children's rank. Because of the leftist property, the  right node will always have the minimal rank.
This means that the path from the root to the right most leaf is at most log(N + 1).

We make a typeclass for accessing the rank:

In [25]:
class HasRank a where
  type Rank a
  rank :: a -> Rank a
  
instance HasRank (LeftistHeap a) where
  type Rank (LeftistHeap a) = Int
  rank Leaf = 0
  rank (Node _ r _ _) = r

Now we can begin implementing the Heap interface for the leftist heap

In [26]:
instance Ord a => Heap (LeftistHeap a) where
  type Elem (LeftistHeap a) = a
  
  isEmpty Leaf = True
  isEmpty _ = False
  
  empty = Leaf
  
  singleton x = Node x 1 Leaf Leaf
  
  merge Leaf heap = heap
  merge heap Leaf = heap
  merge h1@(Node x _ left1 right1)
        h2@(Node y _ left2 right2) =
    if x > y
      then mkNode x left1 (merge right1 h2)
      else mkNode y left2 (merge right2 h1)
    where
      mkNode a heap1 heap2 =
        if rank heap1 <= rank heap2
          then Node a (rank heap1 + 1) heap2 heap1
          else Node a (rank heap2 + 1) heap1 heap2
          
  insert x Leaf = singleton x
  insert x heap = merge (singleton x) heap
  
  decompose Leaf = Nothing
  decompose (Node mx _ l r) = Just (mx, merge l r)

### Verify the Leftist Property

To begin, we'll need type level natural numbers and a way to assert a `<=` relationship

In [27]:
:ext DataKinds
:ext GADTs
:ext TypeOperators

:m Data.Kind

data Nat = Z | S Nat deriving (Eq, Show)

data (<=) :: Nat -> Nat -> Type where
  Base :: 'Z <= 'Z
  Single :: a <= b -> a <= 'S b
  Both :: a <= b -> 'S a <= 'S b

We need to define a singleton to be able to recover values from uninhabitable types (such as `'Z` and `'S _`)

In [28]:
data SNat :: Nat -> Type where
  SZ :: SNat 'Z
  SS :: SNat n -> SNat ('S n)
  
recover :: n <= m -> (SNat n, SNat m)
recover Base = (SZ, SZ)
recover (Single p) = let (n, m) = recover p in (n, SS m)
recover (Both p) = let (n, m) = recover p in (SS n, SS m)

#### Rank encoded leftist heaps

We have the pieces necessary to define a leftist heap where the leftist property is verified at the type level.

In [29]:
newtype Rank n = Rank { unRank :: SNat n }

inc :: Rank rank -> Rank ('S rank)
inc (Rank snat) = Rank $ SS snat

data SafeHeap :: Nat -> Type -> Type where
  Leaf' :: SafeHeap 'Z a
  Node' :: a -> Rank ('S m)             -- Node' data
        -> SafeHeap n a -> SafeHeap m a -- Children
        -> m <= n                       -- Leftist invariant
        -> SafeHeap ('S m) a
        
instance HasRank (SafeHeap n a) where
  type Rank (SafeHeap n a) = Rank n

We are not able to define an instance of `Heap` for this directly because the rank type parameter would result in the type of merge being `merge :: SafeHeap rank a -> SafeHeap rank a -> SafeHeap rank a`. We instead want to be able to merge heaps of different ranks to produce another rank.

The answer is to hide the rank parameter using an existential wrapper.

In [30]:
:ext ExistentialQuantification

data SomeSafeHeap a = forall r. SSH (SafeHeap r a)

-- Allows us to produce our type level assertion using values
lemConnexity :: SNat n -> SNat m -> Either (n <= m) (m <= n)
lemConnexity SZ y = Left $ lemZLEQAll y
lemConnexity x SZ = Right $ lemZLEQAll x
lemConnexity (SS x) (SS y) =
  case lemConnexity x y of
    Left xLEQy -> Left $ Both xLEQy
    Right yLEQx -> Right $ Both yLEQx
    
lemZLEQAll :: SNat n -> 'Z <= n
lemZLEQAll SZ = Base
lemZLEQAll (SS n) = Single $ lemZLEQAll n

instance Ord a => Heap (SomeSafeHeap a) where
  type Elem (SomeSafeHeap a) = a
  
  singleton x = SSH singleton' where
    singleton' = Node' x (Rank $ SS SZ) Leaf' Leaf' Base
    
  merge (SSH Leaf') heap = heap
  merge heap (SSH Leaf') = heap
  merge heap1@(SSH (Node' x _ left1 right1 _))
        heap2@(SSH (Node' y _ left2 right2 _)) =
    if x > y
    then mkNode x (SSH left1) $ merge (SSH right1) heap2
    else mkNode y (SSH left2) $ merge (SSH right2) heap1
    where
    mkNode a (SSH h1) (SSH h2) =
      case lemConnexity (unRank $ rank h1) (unRank $ rank h2) of
        Left r1LEQr2 -> SSH $ Node' a (inc $ rank h1) h2 h1 r1LEQr2
        Right r2LEQr1 -> SSH $ Node' a (inc $ rank h2) h1 h2 r2LEQr1

We wrote a lemma that tells us which child heap has the lower rank and should therefore be on the right.

### Verifying the heap property

Now we want to encode both the rank and the label in our leftist heap type.

In [39]:
import Data.Type.Equality ((:~:)(..))

newtype Label n = Label { unLabel :: SNat n }

data SaferHeap :: Nat -> Nat -> Type where
  Leaf'' :: SaferHeap 'Z 'Z
  Node'' :: Label a -> Rank ('S m)         -- Node Data
         -> SaferHeap n b -> SaferHeap m c -- Children
         -> m <= n                         -- Leftist property
         -> b <= a -> c <= a               -- Heap property
         -> SaferHeap ('S m) a
         
instance HasRank (SaferHeap rank label) where
  type Rank (SaferHeap rank label) = Rank rank
  rank Leaf'' = Rank SZ
  rank (Node'' _ r _ _ _ _ _) = r
         
-- the existential wrapper over which we'll define the Heap instance
data SomeSaferHeap = forall rank label. SSH' (SaferHeap rank label)

-- we need to be able to convert between SNat and Nat because we don't actually store a Nat anywhere
demote :: SNat n -> Nat
demote SZ = Z
demote (SS n) = S (demote n)

-- we'll need an existential wrapper for our Nat singleton
data SomeNat = forall n. SomeNat (SNat n)

promote :: Nat -> SomeNat
promote Z = SomeNat SZ
promote (S n) | SomeNat snat <- promote n = SomeNat (SS snat)

-- We need an intermediate existential wrapper that allows us to postulate the label but still hides the rank
data AlmostSomeSaferHeap a = forall rank. ASSH (SaferHeap rank a)

mkNode :: Label c
       -> AlmostSomeSaferHeap a -> AlmostSomeSaferHeap b
       -> a <= c -> b <= c
       -> AlmostSomeSaferHeap c
mkNode vc (ASSH hA) (ASSH hB) aLEQc bLEQc
  | rA <- rank hA, rB <- rank hB =
  case lemConnexity (unRank rA) (unRank rB) of
    Left arLEQbr  -> ASSH $ Node'' vc (inc rA) hB hA arLEQbr bLEQc aLEQc
    Right brLEQar -> ASSH $ Node'' vc (inc rB) hA hB brLEQar aLEQc bLEQc

type family Max n m :: Nat where
  Max 'Z m = m
  Max n 'Z = n
  Max ('S n) ('S m) = 'S (Max n m)
  
lemMaxOfLEQ :: n <= m -> Max n m :~: m
lemMaxOfLEQ Base = Refl
lemMaxOfLEQ (Both xLEQy) | Refl <- lemMaxOfLEQ xLEQy = Refl
lemMaxOfLEQ (Single xLEQy) =
  case fst $ recover xLEQy of
    SZ -> Refl
    SS _ | Refl <- lemMaxOfLEQ (lemDecLEQ xLEQy) -> Refl
    
lemDecLEQ :: 'S x <= y -> x <= y
--lemDecLEQ (Single a) = Single (LemDecLEQ a)
--lemDecLEQ (Both n) = Single n
lemDecLEQ sxLEQy = uncurry go (recover sxLEQy) sxLEQy
  where
  go :: SNat ('S x) -> SNat y -> 'S x <= y -> x <= y
  go _ SZ _ = error "Impossible case."
  go _ (SS _) (Single leq) = Single (lemDecLEQ leq)
  go (SS SZ) y (Both _) = lemZLEQAll y
  go (SS (SS _)) (SS _) (Both leq) = Both (lemDecLEQ leq)

lemDoubleLEQMax :: n <= l -> m <= l -> Max n m <= l
lemDoubleLEQMax nLEQ1 mLEQ1 =
  let n = fst $ recover nLEQ1
      m = fst $ recover mLEQ1
   in case lemMaxSelective n m of
        Left Refl -> nLEQ1
        Right Refl -> mLEQ1

lemMaxSelective :: SNat n -> SNat m -> Either (Max n m :~: n) (Max n m :~: m)
lemMaxSelective SZ _ = Right Refl
lemMaxSelective _ SZ = Left Refl
lemMaxSelective (SS n) (SS m) =
  case lemMaxSelective n m of
    Left Refl -> Left Refl
    Right Refl -> Right Refl

lemMaxSym :: SNat n -> SNat m -> Max n m :~: Max m n
lemMaxSym SZ _ = Refl
lemMaxSym _ SZ = Refl
lemMaxSym (SS n) (SS m) | Refl <- lemMaxSym n m = Refl

In [40]:
merge' :: AlmostSomeSaferHeap n -> AlmostSomeSaferHeap m -> AlmostSomeSaferHeap (Max n m)
merge' (ASSH Leaf'') heap = heap
merge' heap (ASSH Leaf'') = heap
merge' heap1@(ASSH (Node'' label1 _ left1 right1 _ lLEQ1 rLEQ1))
       heap2@(ASSH (Node'' label2 _ left2 right2 _ lLEQ2 rLEQ2)) =
  case lemConnexity (unLabel label1) (unLabel label2) of
    Left l1LEQl2 | Refl <- lemMaxOfLEQ l1LEQl2 ->
      let child1 = ASSH left2
          c1LEQp = lLEQ2
          child2 = merge' (ASSH right2) heap1
          c2LEQp = lemDoubleLEQMax rLEQ2 l1LEQl2
       in mkNode label2 child1 child2 c1LEQp c2LEQp
    Right l2LEQl1 | Refl <- lemMaxOfLEQ l2LEQl1
                  , Refl <- lemMaxSym (unLabel label1) (unLabel label2) ->
      let child1 = ASSH left1
          c1LEQp = lLEQ1
          child2 = merge' (ASSH right1) heap2
          c2LEQp = lemDoubleLEQMax rLEQ1 l2LEQl1
       in mkNode label1 child1 child2 c1LEQp c2LEQp

instance Heap SomeSaferHeap where
  type Elem SomeSaferHeap = Nat
  
  empty = SSH' Leaf''
  
  singleton x | SomeNat n <- promote x =
    let zLTEn = lemZLEQAll n
     in SSH' $ Node'' (Label n) (Rank $ SS SZ) Leaf'' Leaf'' Base zLTEn zLTEn
     
  merge (SSH' Leaf'') heap = heap
  merge heap (SSH' Leaf'') = heap
  merge (SSH' h1) (SSH' h2) | ASSH m <- merge' (ASSH h1) (ASSH h2) = SSH' m
    

In [35]:
testHeap :: SomeSaferHeap
testHeap = fromList [Z, S Z, S (S Z), Z]

testListHeap :: [Int]
testListHeap = fromList [2 :: Int, 3, 4, 1]
decompose testListHeap

Just (4,[2,3,1])